In [ ]:
# NLP Lecture @ Strive School - 21st July 2021
# NER update

'''
Since today we are exploring the world of natural language processing, we’ll deepen in the Named Entity Recognition technique: this is just one of the mechanisms that NLP embodies. The recognition of named entities as the process of automatic identification of the entities present in a text and consequent classification into predefined categories such as "person", "organization", "position" is a quite common activity and expect for English, trained models with spaCy offer few labels that could be improved through training.

Following the case study of this morning, try to emulate it in order to label all the brands present in the provided datasets, choosing the one you prefer OR trying to label all them and to train the model to recognize new different entities. The result should be twofold: the final model should be able to recognize brands that it has already seen, but already new ones.
The brands proposed in the dataset concern fashion, cars and food.
In order to test the accuracy of the model, test it with sentences and brands the model has never seen.

Sample of the dataset
---------------------
- Cate Blanchett in Armani Privé. Rating: 8. Concludes as a rare butterfly, or from Rorschach's Test, or from computerized axial tomography.
- I liked everything, recommend it! Another quality Xiaomi product...
- What is the price of that Fiat 500XL?

Info:
- Feel free to change or arrange a new dataset
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)

'''

In [6]:
# STEP 0 - PRE REQUISITES

# python -m spacy download en_core_web_lg

# TBD: Import libraries
import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TBD: Load preferred model

nlp = spacy.load("en_core_web_lg")

with open("food.txt") as file:
    dataset = file.read()

# TBD: Load the dataset and test it as-is

doc = nlp(dataset)

print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

Entities: [('Italian', 'NORP'), ('first', 'ORDINAL'), ('Arrosticini', 'PERSON'), ('Alfredo', 'PERSON'), ('Tiramisù', 'PERSON'), ('Naples', 'GPE'), ('Zaza', 'PERSON'), ('ApplePie', 'ORG'), ('Bologna', 'GPE'), ('Fiorentina Steak', 'PERSON'), ('Pineapple', 'ORG'), ('First', 'ORDINAL'), ('Bronte', 'PERSON'), ('Coca-cola', 'ORG'), ('Fanta', 'ORG'), ('Pepsi', 'ORG'), ('One', 'CARDINAL'), ('Sorrento', 'GPE'), ('Coffee\nBread', 'ORG'), ('Love', 'WORK_OF_ART'), ('Fatte', 'PERSON'), ('vedrai che il mondo poi ti sorriderà', 'PERSON'), ('Pastiera', 'PERSON'), ('the United States', 'GPE'), ('Two', 'CARDINAL'), ('two', 'CARDINAL'), ('24 hours', 'TIME'), ('24', 'CARDINAL'), ('two hours', 'TIME')]


In [19]:
# STEP 1 - TRAIN DATA

# Prepare training data

# TBD: define all the entities by extracting the words and their indexes from the dataset
# expected format is the following:  ("sentence", {"entities": [0,10, "FOOD"]})

words = ["ketchup", "pasta", "carrot", "pizza", "garlic", "tomato sauce", "basil", "carbonara", "eggs", 
        "cheek fat", "pancakes", "parmigiana", "eggplant", "fettucine", "heavy cream", "polenta", "risotto", 
        "espresso", "arrosticinni", "spaghetti", "fiorentina steak", "pecorino", "maccherone", "nutella", "amaro", 
        "pistachio", "coca-cola", "wine", "watermelon", "pastiera", "cappuccino", "ice cream", "soup", "lemon", 
        "chocolate", "pineapple"]


train_data = []

with open("food.txt") as file:
    sentences = file.readlines()
    for sent in sentences:
        print("******")
        print("Sentence:", sent)
        print("******")
        sent = sent.lower()
        entities = []
        for word in words:
            word = word.lower()
            if word in sent:
                start_idx = sent.index(word)
                end_idx = len(word) + start_idx
                print("word:", word)
                print("************")
                print("Start Index:", start_idx)
                print("End Index:", end_idx)
                pos = (start_idx, end_idx, "FOOD")
                entities.append(pos)
        element = (sent.strip("\n"), {"entities": entities})

        train_data.append(element)
        print("----------------")
        print("element:", element)


# STEP 2 - UPDATE MODEL

# TBD: load the needed pipeline
ner = nlp.get_pipe("ner")

# TBD: define the annotations

# TBD: train the model

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# TBD: define the number of iterations, the batch size and the drop according to your experience or using an empirical value
# Train model
with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(0):
        print("Iteration #" + str(iteration))

        # Data shuffle for each iteration
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in spacy.util.minibatch(train_data, size=3):
            for text, annotations in batch:
                # Create an Example object
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0)
        print("Losses:", losses)

# Save the model
# TBD:

******
Sentence: Give me carrot cake.

******
word: carrot
************
Start Index: 8
End Index: 14
----------------
element: ('give me carrot cake.', {'entities': [(8, 14, 'FOOD')]})
******
Sentence: I love simple pizza margherita with tomato sauce and mozzarella.

******
word: pizza
************
Start Index: 14
End Index: 19
word: tomato sauce
************
Start Index: 36
End Index: 48
----------------
element: ('i love simple pizza margherita with tomato sauce and mozzarella.', {'entities': [(14, 19, 'FOOD'), (36, 48, 'FOOD')]})
******
Sentence: I don't like pizza with garlic!

******
word: pizza
************
Start Index: 13
End Index: 18
word: garlic
************
Start Index: 24
End Index: 30
----------------
element: ("i don't like pizza with garlic!", {'entities': [(13, 18, 'FOOD'), (24, 30, 'FOOD')]})
******
Sentence: I like pasta with homemade tomato sauce and basil.

******
word: pasta
************
Start Index: 7
End Index: 12
word: tomato sauce
************
Start Index: 27
E

In [ ]:
# STEP 3 - TEST THE UPDATED MODEL

# Load updated model
nlp_updated = spacy.load(output_dir)

# TBD: test with a old sentence
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# TBD: test with a new sentence and an old brand

# TBD: test with a new sentence and a new brand